In [91]:
import math
from collections import Counter
import pickle
import re
import nltk
from nltk.corpus import stopwords
from lyricsgenius import Genius

with open('../lyrics-updated.pkl', 'rb') as f:
    song_lyrics = pickle.load(f)


with open('../classifications.pkl', 'rb') as f1:
    classifications = pickle.load(f1)

with open('../output/big-songs-updated.pkl', 'rb') as f2:
    big_songs = pickle.load(f2)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def tokenize(lyrics):
    words = lyrics.lower().split()
    return [word for word in words if word not in stop_words]

k1 = 1.0
b = 0.5

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\civip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
# call the api to get the lyrics

genius = Genius('HhcPCGMwENzdfMQlauEV1Om0UormkWc6Wrwxri-LFTfBSNkzhwMSVUnR5tZG8eXW')
genius.remove_section_headers = True
genius.skip_non_songs = False
genius.excluded_terms = ["Remix", "Live"]

# song_name = "Happy"
# artist = "Pharrel Williams"
# song_name = "Firework"
# artist = "Katy Perry"
# song_name = "I Hate Everything About You"
# artist = "Three Days Grace"
# song_name = "Coffee"
# artist = "Chappell Roan"
song_name = "Boulevard of Broken Dreams"
artist = "Green Day"


song = genius.search_song(song_name, artist)
pos = song.lyrics.find(f'{song_name} Lyrics')
query = song.lyrics[pos:]



Searching for "Coffee" by Chappell Roan...
Done.


In [93]:
# then remove  capital letters and non alphanumeric characters
query = query.lower()
query = ''.join([char if char.isalpha() or char.isspace() else '' for char in query.replace('\n', ' ')])
# it might seem crazy what im bout to say
query



'coffee lyrics cant meet you for dinner at the italian place its where i met your family some words were exchanged id suggest the jazz bar on mary ann street but youd buy me a drink\u205fand\u205fwe\u205fknow where that\u205fleads so  ill meet\u205fyou for coffee cause if we have wine youll say that you want me i know thats a lie if i didnt love you it would be fine meet you for coffee only for coffee nowhere else is safe every place leads back to your place  you said lets do the park cause i love the park that may be true but god forbid it gets dark here come the excuses that fuel the illusions but id rather feel something than nothing at all so  ill meet you for coffee cause if we have wine youll say that youre sorry i know thats a lie if i didnt trust you it would be fine ill meet you for coffee only for coffee nowhere else is safe every place leads back to your  see chappell roan liveget tickets as low as you might also like weve done this before and i dont need it anymore  so lets

In [94]:
# tokenize all documents
tokenized_songs = {}
for category, all_lyrics in big_songs.items():
    tokenized_songs[category] = tokenize(all_lyrics)

# calculate document lengths
doc_lengths = {}
for  category, all_lyrics in tokenized_songs.items():
    doc_lengths[category] = len(all_lyrics)

# avg doc length
avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)


In [95]:
# calculate doc freqs
doc_freqs = {}
for doc in tokenized_songs.values():
    unique_terms = set(doc)
    for term in unique_terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1

In [96]:
# bm25 algorithm
def bm25(query, document, doc_length):
    score = 0.0
    query_terms = tokenize(query)
    term_freqs = Counter(document)
    
    for term in query_terms:
        if term in doc_freqs:  # term must appear in the corpus
            # Calculate BM25 components
            doc_freq = doc_freqs[term]
            idf = math.log((len(big_songs) - doc_freq + 0.5) / (doc_freq + 0.5) + 1)
            term_freq = term_freqs[term]
            term_score = idf * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * doc_length / avg_doc_length))
            score += term_score
    return score

In [97]:
scores = []
for doc_id, doc in tokenized_songs.items():
    score = bm25(query, doc, doc_lengths[doc_id])
    scores.append((doc_id, score))

# Sort documents by score in descending order
ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

# Display the ranking results
print("Document Rankings:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"Rank {rank}: Document {doc_id} with score {score:.4f}")

Document Rankings:
Rank 1: Document mild with score 26.3897
Rank 2: Document curious with score 23.5736
Rank 3: Document pleasing with score 23.3617
Rank 4: Document electric with score 21.7011
Rank 5: Document hectic with score 21.0720
Rank 6: Document fulfilled with score 19.5661
Rank 7: Document confident with score 19.4973
Rank 8: Document indifferent with score 19.0466
Rank 9: Document passionate with score 18.8336
Rank 10: Document delightful with score 18.7744
Rank 11: Document animated with score 18.1456
Rank 12: Document eager with score 15.6551
Rank 13: Document enthusiastic with score 15.3123
Rank 14: Document determined with score 15.1244
Rank 15: Document lukewarm with score 15.1241
Rank 16: Document neutral with score 15.0750
Rank 17: Document serious with score 14.8654
Rank 18: Document thrilling with score 14.1789
Rank 19: Document frustration with score 14.0713
Rank 20: Document satisfaction with score 14.0211
Rank 21: Document excited with score 13.9074
Rank 22: Docum